In [103]:
import pandas as pd

### Education

We began with multiple different tables including this education one and pivoted the table from having multiple rows for every job and education category to columns representing the percentage of each person in that career field with a certain educational level. 

In [135]:
education = pd.read_csv("Education, Training, and Experience.csv")
education = education[education["Element Name"] == "Required Level of Education"]

# Normalize data
education["Data Value"] = (education["Data Value"]/100).round(3)

# Switch rows to columns
education = education.pivot_table(index=["O*NET-SOC Code", "Title"], 
                                        columns="Category", 
                                        values="Data Value",
                                        aggfunc="first")

# Edit column titles
education.columns = [f"Category {int(col)}" for col in education.columns]
education = education.reset_index()

education

,O*NET-SOC Code,Title,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Category 11,Category 12
0,11-1011.00,Chief Executives,0.000,0.045,0.000,0.000,0.052,0.323,0.000,0.459,0.039,0.006,0.049,0.028
1,11-1011.03,Chief Sustainability Officers,0.000,0.000,0.000,0.000,0.000,0.185,0.000,0.741,0.074,0.000,0.000,0.000
2,11-1021.00,General and Operations Managers,0.104,0.288,0.057,0.216,0.060,0.273,0.001,0.002,0.000,0.000,0.000,0.000
3,11-2011.00,Advertising and Promotions Managers,0.062,0.098,0.000,0.077,0.080,0.600,0.024,0.059,0.000,0.000,0.000,0.000
4,11-2021.00,Marketing Managers,0.000,0.035,0.000,0.028,0.030,0.558,0.000,0.244,0.000,0.105,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,53-7071.00,Gas Compressor and Gas Pumping Station Operators,0.000,0.646,0.104,0.111,0.139,0.000,0.000,0.000,0.000,0.000,0.000,0.000
854,53-7072.00,"Pump Operators, Except Wellhead Pumpers",0.000,0.609,0.084,0.085,0.046,0.176,0.000,0.000,0.000,0.000,0.000,0.000
855,53-7073.00,Wellhead Pumpers,0.013,0.805,0.109,0.000,0.074,0.000,0.000,0.000,0.000,0.000,0.000,0.000
856,53-7081.00,Refuse and Recyclable Material Collectors,0.016,0.815,0.057,0.000,0.112,0.000,0.000,0.000,0.000,0.000,0.000,0.000


#### Education Categories

There was also a specific table for what each category represented in terms of education level, so we wanted to add that to our table for more readability.

In [139]:
education_categories = pd.read_csv("Education, Training, and Experience Categories.csv")
education_categories = education_categories[education_categories["Element Name"] == "Required Level of Education"]
education_categories = education_categories[["Element Name", "Category", "Category Description"]]
education_categories

,Element Name,Category,Category Description
0,Required Level of Education,1,Less than a High School Diploma
1,Required Level of Education,2,High School Diploma - or the equivalent (for e...
2,Required Level of Education,3,Post-Secondary Certificate - awarded for train...
3,Required Level of Education,4,Some College Courses
4,Required Level of Education,5,Associate's Degree (or other 2-year degree)
5,Required Level of Education,6,Bachelor's Degree
6,Required Level of Education,7,Post-Baccalaureate Certificate - awarded for c...
7,Required Level of Education,8,Master's Degree
8,Required Level of Education,9,Post-Master's Certificate - awarded for comple...
9,Required Level of Education,10,First Professional Degree - awarded for comple...


We decided for the purpose of our job recommendation system to be slightly vaguer than the data regarding education levels so we combined some of the categories as well as the data for each of them.

In [141]:
# Combine 12 categories into 4 broader categories to make data more concise

combined_education_categories = education_categories.loc[[0,4,5,6]]
combined_education_categories['Category Description'] = ["High School Diploma or Less", 
                                                        "Post-Secondary Certificate, Some College, or Associate's Degree",
                                                        "Bachelor's Degree",
                                                        "Postgraduate Degree"]
combined_education_categories['Category'] = [1, 2, 3, 4]


combined_education_categories

,Element Name,Category,Category Description
0,Required Level of Education,1,High School Diploma or Less
4,Required Level of Education,2,"Post-Secondary Certificate, Some College, or A..."
5,Required Level of Education,3,Bachelor's Degree
6,Required Level of Education,4,Postgraduate Degree


In [144]:
# New Education table with combined Categories

education = education.copy()

# Summed data values to correspond to 4 broader categories
education["Category 1"] = education[["Category 1", "Category 2"]].sum(axis=1)
education["Category 2"] = education[["Category 3", "Category 4", "Category 5"]].sum(axis=1)
education["Category 3"] = education["Category 6"]
education["Category 4"] = education[["Category 7", "Category 8", "Category 9", "Category 10", "Category 11", "Category 12"]].sum(axis=1)

# Select relevant columns 
education = education[["O*NET-SOC Code", "Title", "Category 1", "Category 2", "Category 3", "Category 4"]]

# Rename column titles to match new categories
education.rename(columns={"Category 1":"High School Diploma or Less",
                         "Category 2":"Post-Secondary Certificate, Some College, or Associate's Degree",
                         "Category 3":"Bachelor's Degree",
                         "Category 4":"Postgraduate Degree"},inplace=True)

education

,O*NET-SOC Code,Title,High School Diploma or Less,"Post-Secondary Certificate, Some College, or Associate's Degree",Bachelor's Degree,Postgraduate Degree
0,11-1011.00,Chief Executives,0.045,0.052,0.323,0.581
1,11-1011.03,Chief Sustainability Officers,0.000,0.000,0.185,0.815
2,11-1021.00,General and Operations Managers,0.392,0.333,0.273,0.003
3,11-2011.00,Advertising and Promotions Managers,0.160,0.157,0.600,0.083
4,11-2021.00,Marketing Managers,0.035,0.058,0.558,0.349
...,...,...,...,...,...,...
853,53-7071.00,Gas Compressor and Gas Pumping Station Operators,0.646,0.354,0.000,0.000
854,53-7072.00,"Pump Operators, Except Wellhead Pumpers",0.609,0.215,0.176,0.000
855,53-7073.00,Wellhead Pumpers,0.818,0.183,0.000,0.000
856,53-7081.00,Refuse and Recyclable Material Collectors,0.831,0.169,0.000,0.000


### Work Values

We also had to pivot the work values table as there was a row for each of the values that correlate with each job. After, we normalized the values for each to be from the range of 0 to 1 instead of 0 to 7 to make the data more understandable and easier to use for our model.

In [116]:
work_values = pd.read_csv("Work Values.csv")

# Normalize data
work_values["Data Value"] = (work_values["Data Value"]/7).round(3)

# Select relevant work_value/scale
work_values = work_values[work_values["Scale ID"] == "EX"]

# Switch rows to columns
work_values = work_values.pivot_table(index=["O*NET-SOC Code", "Title"], 
                                        columns="Element Name", 
                                        values="Data Value",
                                        aggfunc="first")
work_values.columns = [f"Values {col}" for col in work_values.columns]
work_values

,,Values Achievement,Values Independence,Values Recognition,Values Relationships,Values Support,Values Working Conditions
O*NET-SOC Code,Title,,,,,,
11-1011.00,Chief Executives,0.904,1.000,1.000,0.714,0.761,0.904
11-1011.03,Chief Sustainability Officers,0.953,0.953,0.857,0.714,0.476,0.904
11-1021.00,General and Operations Managers,0.761,0.857,0.810,0.904,0.667,0.857
11-1031.00,Legislators,0.761,0.714,0.714,0.810,0.571,0.619
11-2011.00,Advertising and Promotions Managers,0.761,0.761,0.761,0.714,0.571,0.761
...,...,...,...,...,...,...,...
53-7071.00,Gas Compressor and Gas Pumping Station Operators,0.333,0.524,0.286,0.476,0.667,0.476
53-7072.00,"Pump Operators, Except Wellhead Pumpers",0.333,0.524,0.333,0.524,0.810,0.453
53-7073.00,Wellhead Pumpers,0.333,0.429,0.333,0.619,0.810,0.453


### Interests

We also had to pivot the interests table as there was a row for each of the interests that correlate with each job. After, we normalized the values for each to be from the range of 0 to 1 instead of 0 to 7 to make the data more understandable and easier to use for our model.

In [119]:
interests = pd.read_csv("Interests.csv")

# Normalize data
interests["Data Value"] = (interests["Data Value"]/7).round(3)

# Select relevant Interest/scale
interests = interests[interests["Scale ID"] == "OI"]
interests = interests.pivot_table(index=["O*NET-SOC Code", "Title"], 
                                        columns="Element Name", 
                                        values="Data Value",
                                        aggfunc="first")

# Set column titles
interests.columns = [f"Interest in {col} Aspect" for col in interests.columns]

interests

,,Interest in Artistic Aspect,Interest in Conventional Aspect,Interest in Enterprising Aspect,Interest in Investigative Aspect,Interest in Realistic Aspect,Interest in Social Aspect
O*NET-SOC Code,Title,,,,,,
11-1011.00,Chief Executives,0.297,0.714,0.983,0.463,0.186,0.503
11-1011.03,Chief Sustainability Officers,0.354,0.641,0.954,0.683,0.291,0.507
11-1021.00,General and Operations Managers,0.187,0.760,0.994,0.341,0.317,0.481
11-1031.00,Legislators,0.386,0.517,0.789,0.479,0.220,0.527
11-2011.00,Advertising and Promotions Managers,0.550,0.614,1.000,0.244,0.153,0.449
...,...,...,...,...,...,...,...
53-7071.00,Gas Compressor and Gas Pumping Station Operators,0.143,0.693,0.217,0.333,0.877,0.220
53-7072.00,"Pump Operators, Except Wellhead Pumpers",0.143,0.644,0.209,0.323,0.953,0.183
53-7073.00,Wellhead Pumpers,0.143,0.623,0.267,0.344,0.967,0.153


### Occupations

This is the occupation table with the code associated with each job title and the description to provide users when given the reccomendations.

In [122]:
occupations = pd.read_csv("Occupation Data.csv")
occupations

,O*NET-SOC Code,Title,Description
0,11-1011.00,Chief Executives,Determine and formulate policies and provide o...
1,11-1011.03,Chief Sustainability Officers,"Communicate and coordinate with management, sh..."
2,11-1021.00,General and Operations Managers,"Plan, direct, or coordinate the operations of ..."
3,11-1031.00,Legislators,"Develop, introduce, or enact laws and statutes..."
4,11-2011.00,Advertising and Promotions Managers,"Plan, direct, or coordinate advertising polici..."
...,...,...,...
1011,55-3014.00,Artillery and Missile Crew Members,"Target, fire, and maintain weapons used to des..."
1012,55-3015.00,Command and Control Center Specialists,"Operate and monitor communications, detection,..."
1013,55-3016.00,Infantry,Operate weapons and equipment in ground combat...
1014,55-3018.00,Special Forces,"Implement unconventional operations by air, la..."


### Combined Table

Finally, we combined all of our normalized tables to use for the model and removed any careers with insufficient data.

In [128]:
# Join data from Education, Work Values, and Interest tables on O*NET_SOC Code 
merged_df = occupations.merge(education, on='O*NET-SOC Code', how='left').merge(work_values, on='O*NET-SOC Code', how='left').merge(interests, on='O*NET-SOC Code', how='left')

# Edit table
merged_df = merged_df.drop(["Description","Title_y"], axis = 1)
merged_df = merged_df.rename(columns={"Title_x": "Title"})

# Drop rows with null values
merged_df = merged_df.dropna()
merged_df

,O*NET-SOC Code,Title,High School Diploma or Less,"Post-Secondary Certificate, Some College, or Associate's Degree",Bachelor's Degree,Postgraduate Degree,Values Achievement,Values Independence,Values Recognition,Values Relationships,Values Support,Values Working Conditions,Interest in Artistic Aspect,Interest in Conventional Aspect,Interest in Enterprising Aspect,Interest in Investigative Aspect,Interest in Realistic Aspect,Interest in Social Aspect
0,11-1011.00,Chief Executives,0.045,0.052,0.323,0.581,0.904,1.000,1.000,0.714,0.761,0.904,0.297,0.714,0.983,0.463,0.186,0.503
1,11-1011.03,Chief Sustainability Officers,0.000,0.000,0.185,0.815,0.953,0.953,0.857,0.714,0.476,0.904,0.354,0.641,0.954,0.683,0.291,0.507
2,11-1021.00,General and Operations Managers,0.392,0.333,0.273,0.003,0.761,0.857,0.810,0.904,0.667,0.857,0.187,0.760,0.994,0.341,0.317,0.481
4,11-2011.00,Advertising and Promotions Managers,0.160,0.157,0.600,0.083,0.761,0.761,0.761,0.714,0.571,0.761,0.550,0.614,1.000,0.244,0.153,0.449
5,11-2021.00,Marketing Managers,0.035,0.058,0.558,0.349,0.857,0.810,0.761,0.810,0.714,0.881,0.350,0.671,1.000,0.414,0.143,0.403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,53-7071.00,Gas Compressor and Gas Pumping Station Operators,0.646,0.354,0.000,0.000,0.333,0.524,0.286,0.476,0.667,0.476,0.143,0.693,0.217,0.333,0.877,0.220
992,53-7072.00,"Pump Operators, Except Wellhead Pumpers",0.609,0.215,0.176,0.000,0.333,0.524,0.333,0.524,0.810,0.453,0.143,0.644,0.209,0.323,0.953,0.183
993,53-7073.00,Wellhead Pumpers,0.818,0.183,0.000,0.000,0.333,0.429,0.333,0.619,0.810,0.453,0.143,0.623,0.267,0.344,0.967,0.153
994,53-7081.00,Refuse and Recyclable Material Collectors,0.831,0.169,0.000,0.000,0.286,0.333,0.190,0.571,0.619,0.404,0.143,0.630,0.221,0.253,1.000,0.199
